## Une solution commerciale complète

## DÉFI COMMERCIAL

Créez un produit qui génère une brochure pour une entreprise à utiliser pour des clients potentiels, des investisseurs et des recrues potentielles.

Nous fournirons un nom d'entreprise et leur site Web principal. Consultez la fin de ce carnet pour des exemples d'applications commerciales réelles.


In [2]:
# imports
# Si cela échoue, veuillez vérifier que vous exécutez depuis un environnement 'activé' avec (llms) dans l'invite de commande.

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from langdetect import detect

In [3]:
# Initialiser et constantes

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("La clé API a l'air bonne jusqu'à présent")
else:
    print("Il pourrait y avoir un problème avec votre clé API ? Veuillez consulter le carnet de dépannage!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clé API a l'air bonne jusqu'à présent


In [4]:
# Une classe pour représenter une page Web

# Certains sites web nécessitent que vous utilisiez des en-têtes appropriés lors de leur récupération :
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    Une classe utilitaire pour représenter un site Web que nous avons extrait, maintenant avec des liens.
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
press = Website("https://wordpress.com/fr/")
press.links

['https://wordpress.com/fr/',
 'https://wordpress.com/fr/pricing/',
 'https://wordpress.com/log-in/fr',
 'https://wordpress.com/setup/onboarding/fr?ref=logged-out-homepage-lp',
 'https://wordpress.com/fr/hebergement/',
 'https://wordpress.com/fr/pour-les-agences/',
 'https://wordpress.com/fr/affilies/',
 'https://wordpress.com/fr/domaines/',
 'https://wordpress.com/fr/ai-website-builder/?ref=topnav',
 'https://wordpress.com/fr/createur-de-site-web/',
 'https://wordpress.com/fr/create-blog/',
 'https://wordpress.com/fr/newsletter/',
 'https://wordpress.com/fr/e-mail-pro/',
 'https://wordpress.com/fr/service-de-conception-de-site-web/',
 'https://wordpress.com/fr/ecommerce/',
 'https://wpvip.com/wordpress-vip-agile-content-platform?utm_source=WordPresscom&utm_medium=automattic_referral&utm_campaign=top_nav',
 'https://wordpress.com/fr/fonctionnalites/',
 'https://wordpress.com/fr/themes',
 'https://wordpress.com/fr/plugins',
 'https://wordpress.com/fr/patterns',
 'https://wordpress.com/f

## Utilisez un appel à gpt-4o-mini pour lire les liens sur une page web et répondre en JSON structuré. 

Il doit décider quels liens sont pertinents et remplacer les liens relatifs tels que "/about" par "https://company.com/about". 
Nous utiliserons le "one shot prompting" dans lequel nous fournissons un exemple de la façon dont il devrait répondre dans l'invite. C'est un excellent cas d'utilisation pour un LLM, car cela nécessite une compréhension nuancée. 
Imaginez essayer de coder cela sans LLM en analysant et en analysant la page web - ce serait très difficile ! la semaine 8 de notre projet d'intelligence artificielle autonome Agentic.

In [6]:
link_system_prompt = "Vous disposez d'une liste de liens trouvés sur une page web. \
Vous pouvez décider lesquels des liens seraient les plus pertinents à inclure dans une brochure sur l'entreprise, \
tels que des liens vers une page À propos, une page Entreprise, ou des pages Carrières/Emploi.\n"
link_system_prompt += "Vous devriez répondre en JSON comme dans cet exemple:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

Vous disposez d'une liste de liens trouvés sur une page web. Vous pouvez décider lesquels des liens seraient les plus pertinents à inclure dans une brochure sur l'entreprise, tels que des liens vers une page À propos, une page Entreprise, ou des pages Carrières/Emploi.
Vous devriez répondre en JSON comme dans cet exemple:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Voici la liste des liens sur le site web de {website.url} - "
    user_prompt += "Veuillez décider lesquels de ces liens web sont pertinents pour une brochure sur l'entreprise, répondez avec l'URL complète en format JSON. \
N'incluez pas les conditions d'utilisation, la politique de confidentialité, les liens par e-mail.\n"
    user_prompt += "Liens (certains pourraient être des liens relatifs):\n"
    user_prompt += "\n".join(website.links) # ajoute la liste des liens extrait de ce site web, chacun sur une nouvelle ligne.
    return user_prompt

In [10]:
print(get_links_user_prompt(press))

Voici la liste des liens sur le site web de https://wordpress.com/fr/ - Veuillez décider lesquels de ces liens web sont pertinents pour une brochure sur l'entreprise, répondez avec l'URL complète en format JSON. N'incluez pas les conditions d'utilisation, la politique de confidentialité, les liens par e-mail.
Liens (certains pourraient être des liens relatifs):
https://wordpress.com/fr/
https://wordpress.com/fr/pricing/
https://wordpress.com/log-in/fr
https://wordpress.com/setup/onboarding/fr?ref=logged-out-homepage-lp
https://wordpress.com/fr/hebergement/
https://wordpress.com/fr/pour-les-agences/
https://wordpress.com/fr/affilies/
https://wordpress.com/fr/domaines/
https://wordpress.com/fr/ai-website-builder/?ref=topnav
https://wordpress.com/fr/createur-de-site-web/
https://wordpress.com/fr/create-blog/
https://wordpress.com/fr/newsletter/
https://wordpress.com/fr/e-mail-pro/
https://wordpress.com/fr/service-de-conception-de-site-web/
https://wordpress.com/fr/ecommerce/
https://wpvip

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Nous allons utiliser le site HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1-0528',
 '/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B',
 '/ResembleAI/chatterbox',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/models',
 '/spaces/ResembleAI/Chatterbox',
 '/spaces/enzostvs/deepsite',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/wushuang98/Direct3D-S2-v1.0-demo',
 '/spaces',
 '/datasets/open-r1/Mixture-of-Thoughts',
 '/datasets/yandex/yambda',
 '/datasets/MiniMaxAI/SynLogic',
 '/datasets/cognitivecomputations/china-refusals',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writ

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Brochure

Assemblez tous les détails dans une autre invite pour GPT4-o

In [13]:
def detect_website_language(url):
    try:
        text = Website(url).get_contents()
        
        # Détection de la langue
        langue = detect(text)
        return langue
    except Exception as e:
        print(f"Erreur lors du traitement de l'URL {url} : {e}")
        return None
        

In [16]:
url = "https://fr.wikipedia.org/wiki/%C3%89lection_pr%C3%A9sidentielle_en_France"
detect_website_language(url)

'fr'

In [21]:
def get_all_details(url):
    check = detect_website_language(url)
    if check == 'fr':    
        result = "Page d’accueil:\n"
    else:
        result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    if check == 'fr':
        print("Liens trouvés:", links)
    else:
        print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [22]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-R1-0528
Updated
4 days ago
•
39.5k
•
1.59k
deepseek-ai/DeepSeek-R1-0528-Qwen3-8B
Updated
4 days ago
•
41.7k
•
549
ResembleAI/chatterbox
Updated
3 days ago

In [32]:
system_prompt1 = "Vous êtes un assistant qui analyse le contenu de plusieurs pages pertinentes d'un site web d'entreprise \
et crée une courte brochure sur l'entreprise pour les clients potentiels, les investisseurs et les recrues. Réponds en markdown.\
Incluez des détails sur la culture de l'entreprise, les clients et les carrières/emplois si vous avez cette information."

# Ou décommentez les lignes ci-dessous pour une brochure plus humoristique - cela montre à quel point il est facile d'incorporer le 'ton':

system_prompt2 = "Vous êtes un assistant qui analyse le contenu de plusieurs pages pertinentes d'un site web d'entreprise \
 et crée une brochure courte, humoristique, divertissante et pleine de blagues sur l'entreprise pour les clients potentiels, les investisseurs et les recrues. Répondez en markdown.\
 Incluez des détails sur la culture de l'entreprise, les clients et les carrières/emplois si vous avez cette information."

system_prompt = system_prompt2


In [25]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Vous regardez une entreprise appelée: {company_name}\n"
    user_prompt += f"Voici le contenu de sa page d'accueil et d'autres pages pertinentes ; utilisez ces informations pour créer une courte brochure de l'entreprise en markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Tronquer si plus de 5 000 caractères
    return user_prompt

In [34]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


"Vous regardez une entreprise appelée: HuggingFace\nVoici le contenu de sa page d'accueil et d'autres pages pertinentes ; utilisez ces informations pour créer une courte brochure de l'entreprise en markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1-0528\nUpdated\n4 days ago\n•\n41.6k\n•\n1.62k\ndeepseek-ai/DeepSeek-R1-0528-Qwen3-8B\nUpdated\n4 days ago\n•\n55.8k\n•\n571\nResembleAI/chatterbox\nUpdated\n3 days ago\n•\n495\ngoogle/gemma-3n-E4B-it-litert-preview\nUpdated\n7 days ago\n•\n795\nByteDance-Seed/BAGEL-7B-MoT\nUpdated\n11 days ago\n•\n8.22k\n•\n917\nBrowse 1M+ models\nSpaces\nRunning\non\n

In [35]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [36]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}


```markdown
# 🐻‍❄️ Brochure Hugging Face: L'Hug du Futur en IA! 

Bienvenue dans le monde doux comme un câlin de Hugging Face ! 🌟

## Qui est Hugging Face ? 🤔
🚀 **Hugging Face** est l’entreprise où la communauté de l'intelligence artificielle se réunit pour bâtir le futur… en gros, on joue à RoboCop, mais avec des câlins ! Nous fournissons une plateforme pour collaborer sur des modèles, des ensembles de données et des applications d'IA, où les machines apprennent à parler, écrire et même... créer des vidéos (ne vous inquiétez pas, pas de machines remplaçant les acteurs de sitôt !).

## Ce Qu'on Fait 🔍
- ⚙️ **Modèles** : Avec plus de 1M de modèles, vous pouvez enfin trouver celui qui correspond à votre quête existentielle de la machine apprenante parfaite.
- 📊 **Datasets** : Besoin de données? Nous avons 250k+ ensembles de données, parce que pourquoi pas !
- 🌌 **Espaces** : Exprimez-vous avec nos applications - transformez vos idées en réalité (vous pouvez même faire des vidéos à partir d'images statiques... c'est presque de la magie !).

## Nos Clients 🚀
Plus de **50,000 organisations** adorent utiliser Hugging Face, y compris des géants comme **Google, Microsoft, Amazon**, et même de la petite entreprise du coin qui cherche à ne pas rater le virage numérique ! 

## La Culture d'Entreprise 🎉
Chez Hugging Face, la collaboration est dans notre ADN, un peu comme le chocolat dans un brownie. Nous avons une ambiance où le partage des idées, l'innovation, et des blagues sur des modèles de langage flamboyants sont au rendez-vous. 

### Travailler avec Nous 💼
Vous cherchez un emploi où vos idées peuvent faire un câlin à la technologie ? Rejoignez notre équipe ! Que vous soyez un magicien des données ou un développeur d'applications, nous recherchons des esprits brillants avec un sens de l’humour (obligatoire) pour façonner l’avenir de l’IA ensemble.

## Pourquoi Choisir Hugging Face ? 🤗
- Accès à des solutions **Compute** incroyables (à partir de 0,60 $/heure !).
- **Support prioritaire** pour les utilisateurs tout en gardant un esprit ludique.
- Système de sécurité de niveau entreprise, pas de problème, même si votre modèle est un peu... capricieux. 

## En Résumé 🤩
Rejoignez la communauté IA qui construit l’avenir ! Hugging Face n’est pas qu’une simple plateforme ; c’est une Hugging-famille où chaque baiser numérique compte. Alors préparez-vous à plonger dans les océans de l’IA 😊.

**On vous attend avec un grand sourire et un chat sur l’IA!**

Pour plus d'infos, visitez [Hugging Face](https://huggingface.co) et signez notre pétition pour des câlins plus fréquents au bureau. 🐻‍❄️💼

---
```
Ce ton léger et humoristique associe les détails factuels avec un charme engageant, promouvant ainsi l’identité unique de Hugging Face tout en attirant les clients, les investisseurs et les recrues.

## Ameloiration de la memoire

Message renvoyé par OpenAI avec l'animation de machine à écrire familière (progressivement).

In [37]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [38]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Brochure de Hugging Face

## Bienvenue chez Hugging Face - L'entreprise qui vous fait dire "Hug me, AI !"

### Qui sommes-nous ?
Nous sommes une communauté passionnée d'intelligence artificielle, décidée à bâtir le futur. Oui, vous avez bien entendu ! L'avenir n'a jamais été aussi mignon grâce à nos modèles et applications de machine learning !

#### Notre mission ? 
Faciliter la collaboration autour de **modèles, de datasets** et d'**applications**. En gros, nous sommes les "matchmakers" de l'IA !

---

### Services & Offres

- **1M+ de modèles** : C'est comme un buffet à volonté, mais pour les développeurs d'IA ! Mangez à votre faim... euh, codez à votre rythme ! 
- **Applications variées** : Explorez plus de **400k applications**. Vous êtes plutôt text-to-speech ou générateur de vidéos animées ? Nous avons ce qu'il vous faut !

---

### Culture d'entreprise

Chez Hugging Face, nous croyons fermement que la **collaboration** est la clé. Cela signifie que notre bureau (virtuel) est toujours ouvert ! En plus, nos équipes organisent des jeux de société hebdomadaires - parce que rien ne renforce plus une équipe qu'un bon vieux Monopoly (ou pas, mais au moins on rigole !).

#### Rejoignez la Famille !
Nous recherchons des personnes passionnées, curieuses et prêtes à plonger dans l'univers de l'IA. Nos postes sont comme nos modèles : très recherchés ! Et nous vous promettons des rires, des brainstormings (et des blagues sur les robots).

---

### Nos Clients

Plus de **50,000 organisations** font confiance à Hugging Face. Des géants comme Google, Amazon et Microsoft se fient à notre expertise. Est-ce que ça ne vous donne pas envie de faire votre propre part du gâteau ? 🍰

---

### Pourquoi Hugging Face ?

- **Soutien et meilleures pratiques** : Avec notre plateforme, vous ne serez jamais seul dans votre mission de développement.
- **Tarification** : Commencez à partir de **0,60 $/heure pour le GPU**. Qui a dit qu'on ne pouvait pas innover sans se ruiner ?

---

### Prêts pour l'aventure ?

Inscrivez-vous dès aujourd'hui et rejoignez notre communauté grandissante ! 

Et si vous avez des questions, n'oubliez pas : chez Hugging Face, nous avons toujours une **réponse AI**... ou du moins une blague prête à être partagée !

---

### Qu'est-ce qui est mignon et intelligent à la fois ? 

L'IA de Hugging Face, bien sûr ! 

Pour plus d'infos, rendez-vous sur [HuggingFace.co](https://huggingface.co).

---

*N'oubliez pas, chez Hugging Face, il y a toujours une place pour vous... à condition que vous aimiez les câlins !* 🥰

## Utilisation pour le site wordpress 


In [39]:
stream_brochure("wordpress", "https://wordpress.com/fr/")

Liens trouvés: {'links': [{'type': 'about page', 'url': 'https://wordpress.com/fr/about/'}, {'type': 'careers page', 'url': 'https://automattic.com/work-with-us/'}, {'type': 'company page', 'url': 'https://wordpress.com/fr/presse/'}]}



# Bienvenue chez WordPress.com : L'univers des sites web, mais sans les insectes !

### Créez, Vendez, Bloguez et Évitez les Mal de Tête
Que vous soyez un artisan en herbe avec des rêves de grandeur ou un CEO qui a le futur en tête, WordPress.com est là pour vous! Créez un site à couper le souffle, lancez un blog qui pourrait passer pour un chef-d'œuvre littéraire (vous pouvez nous remercier plus tard) ou vendez vos produits et faites de vos rêves une réalité.

### Pourquoi Choisir WordPress.com ?
- **Une bande passante non mesurée** : Comme une bonne pizza, plus on est de fous, plus on rit! Vos visiteurs arriveront à la vitesse de l’éclair. 🚀
- **Sécurité intransigeante** : Pare-feu, chiffrement, protection DDoS… on assure votre tranquillité d’esprit, même si votre mot de passe est "12345". (Spoiler : ne le faites pas!)
- **Créateur de site Web IA** : Pour ceux qui aiment parler à des robots! 🤖 Laissez l'IA se charger des tâches ingrates pendant que vous vous concentrez sur l'essentiel (comme choisir la bonne couleur pour vos boutons).

### Nos Clients : De la Startup à Facebook
Saviez-vous que nous avons des clients prestigieux comme Salesforce, Al Jazeera, Capgemini, et même Facebook? Oui, ils ont tous choisi la plateforme WordPress VIP pour une performance à grande échelle. Et vous? Prêt à nous rejoindre dans cette aventure? (Démarrez à partir de 25,000 $/an, ou cherchez dans votre canapé des pièces de monnaie!)

### La Culture de l'Entreprise : Coolitude et Créativité
Chez WordPress.com, nous croyons fermement en un Web ouvert ! En tant que contributeur principal de la plateforme WordPress (qui alimente plus de 43 % d'Internet, rien que ça), nous sommes fiers de notre culture d’innovateurs, de créateurs, et surtout, de blagueurs ! 

### Carrières chez WordPress : Postulez ou Regrettez 
Envie de travailler avec nous? Préparez vos CV, car nous avons toujours besoin de nouveaux talents ! Que vous soyez un développeur, un designer, ou même un SEO ninja, il y a peut-être une place pour vous à la table de l'innovation. Apportez votre sens de l'humour, nous nous occupons du reste !

### En Résumé:
Si vous visitez WordPress.com, préparez-vous à prendre des décisions OBJECTIVEMENT ASTUCIEUSES! Nous vous offrons une plateforme qui est tellement facile à utiliser que même votre animal de compagnie pourrait créer un blog… ok, peut-être pas, mais vous voyez l'idée!

**En route pour le succès avec WordPress.com - Parce qu’un bon site, c'est comme une bonne blague : il doit faire mouche!**

